In [ ]:
# For visualisation purpose only (off topic)
from graph import draw_dot

# Lesson 16

 - Start with `class Value` skeleton
   - `__init__` without `self._backward`
   - `__repr__` for `print`ing
   - `__add__`, `__mul__`, `__pow__`, `relu`, `__neg__` forward pass only
 - def leaves,
 - compute
   - `s = W @ x`
   - `h = (s)⁺`
   - `y~ = wᵀh`
   - `C = (y - y~)²`
 - draw the tree
 - set `∇C = 1` (here `∇C` is a shorthand for `∂L/∂C`)
 - add `_backward()` to predefined functions to compute `gradInput` given `gradOutput`
 - set `∇C = 1`
 - call `C._backward()` and to all preceding `Values`
 - it takes forever => define `backward()` method
   - sorts topologically the tree
   - set `∇C = 1`
   - iteratively call `_backward()` on `reversed` sorted tree 

 ## Definitions
 - _neuron_: a `list` of weights that compute `s` given an input
 - _layer_: a `list` of neurons that compute `s1`, `s2`, …
 - _model_: a `list` of layers that compute `y~`

 ## Training the model
 `for (x,y) in 𝒟:`
 1. `y~ = model(x)`
 2. `L = C(y, y~)`
 3. `model.zero_grad()` (more about this in the next episode)
 4. `L.backward()`
 5. `w = w - η∂L/∂w`

## Unanswered question
Do we compute or _accumulate_ gradients in `_backward()`?

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label='', color=None):
        self.data = data
        self.grad = 0
        self._prev = _children
        self._op = _op
        self.label = label
        self.color = color
        self._backward = lambda: None
    
    def __repr__(self):
        return f'{self.label or 'Value'}(data={self.data:.2f}, grad={self.grad:.3f})'
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad = out.grad
            other.grad = out.grad
            self.color = 'green'
            other.color = 'green'
        out._backward = _backward

        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '×')

        def _backward():
            self.grad = other.data * out.grad
            other.grad = self.data * out.grad
            self.color = 'green'
            other.color = 'green'
        out._backward = _backward

        return out
    
    def __pow__(self, other):
        out = Value(self.data**other, (self,), f'^{other}')

        def _backward():
            self.grad = (other * self.data**(other-1)) * out.grad
            self.color = 'green'
        out._backward = _backward

        return out
    
    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), '(⋅)⁺')

        def _backward():
            self.grad = (out.data > 0) * out.grad
            self.color = 'green'
        out._backward = _backward

        return out
    
    def __neg__(self): # -self
        return self * Value(-1)
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            v._backward()


In [ ]:
# Leaves
x1  = Value(+1,   label='x1')
x2  = Value(+0,   label='x2')
w11 = Value(-2,   label='w11')
w12 = Value(+4,   label='w12')
w21 = Value(+3,   label='w21')
w22 = Value(-1,   label='w22')
w1  = Value(-1,   label='w1')
w2  = Value(1/12, label='w2')
y   = Value(+1,   label='y')

# s = W @ x
w11x1 = w11 * x1;    w11x1.label = 'w11x1'
w12x2 = w12 * x2;    w12x2.label = 'w12x2'
s1 = w11x1 + w12x2;  s1.label = 's1'
w21x1 = w21 * x1;    w21x1.label = 'w21x1'
w22x2 = w22 * x2;    w22x2.label = 'w22x2'
s2 = w21x1 + w22x2;  s2.label = 's2'

# h = (s)⁺
h1 = s1.relu();  h1.label = 'h1'
h2 = s2.relu();  h2.label = 'h2'

# y~ = wᵀh
w1h1 = w1 * h1;      w1h1.label = 'w1h1'
w2h2 = w2 * h2;      w2h2.label = 'w2h2'
ytld = w1h1 + w2h2;  ytld.label = 'ỹ'

# C = (y - y~)²
_ytld = -ytld;       _ytld.label = '−ỹ'
y_ytld = y + _ytld;  y_ytld.label = 'y − ỹ'
C = y_ytld**2;       C.label = 'C';  C.color = 'red'

In [ ]:
# Draw tree
draw_dot(C)

The next cells need to be run backward, from `C.grad = 1` upwards

In [ ]:
w21x1._backward()

In [ ]:
w22x2._backward()

In [ ]:
s2._backward()

In [ ]:
h2._backward()

In [ ]:
w2h2._backward()

In [ ]:
h1._backward()

In [ ]:
w1h1._backward()

In [ ]:
ytld._backward()

In [ ]:
_ytld._backward()

In [ ]:
y_ytld._backward()

In [ ]:
C._backward()

In [ ]:
# Starts with ∂L/∂C = 1
C.grad = 1

Automating these manual calls

In [ ]:
topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(C)

In [ ]:
topo

In [ ]:
C.backward()